What i want to do: 
- read in postcode dataset
- loop through the code point files 
- join postcodes in each file 
- make sure these are saved and each time loop through a file it doesn't delete earlier merges. 

In [ ]:
# Set libraries
import pandas as pd
import glob
import os
import numpy as np
import datetime
import string
import re

In [ ]:
# Read in postcode dataset
SWW_postcode = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/SWW postcodes.csv')

- Get rid of the white space in the postcodes that have XXXX XXX to XXXXXXX characters so that they match with the OS codepoint dataset

In [ ]:
dtypeCount =[SWW_postcode.iloc[:,i].apply(type).value_counts() for i in range(SWW_postcode.shape[1])]

In [ ]:
dtypeCount

In [ ]:
# Change the postcode field to string
SWW_postcode['postcode']= SWW_postcode['postcode'].astype('str')

In [ ]:
mask = (SWW_postcode['postcode'].str.len()>7)
mask2 = (SWW_postcode['postcode'].str.len()<8)

In [ ]:
df1 = SWW_postcode.loc[mask]

In [ ]:
df2 = SWW_postcode.loc[mask2]

In [ ]:
df2

In [ ]:
SWW_postcode

In [ ]:
df1['First']= df1['postcode'].str[:4]

In [ ]:
df1['Last']= df1['postcode'].str[5:8]

In [ ]:
df1['New_pstcd'] = df1['First'].map(str) + df1['Last']

In [ ]:
df1

In [ ]:
df1_n = df1[['date', 'New_pstcd']]

In [ ]:
df1_n

In [ ]:
df1_n = df1_n.rename(columns={'New_pstcd':'postcode'})

In [ ]:
df_all = pd.concat([df1_n, df2])

In [ ]:
df_all

In [ ]:
df_all = df_all.set_index('postcode')

In [ ]:
df_all

- Create a list of the files within the codepoint folder (containing all the different postcodes)

In [ ]:
files = glob.glob(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/*')

In [ ]:
files

- read in the header file for the Code point postcode files 
- set the index to the column which says postcode

In [ ]:
labels = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Doc/Code-Point_Open_Column_Headers.csv', header = None)

In [ ]:
labels

In [ ]:
labels = labels.set_index([0])

In [ ]:
labels

In [ ]:
for file in files:
    postcode = pd.read_csv(r'{fname}'.format(fname=file), header=None)
    postcode = postcode.set_index([0])
    data = pd.concat([labels,postcode])
    data.columns = data.iloc[0]
    data = data.reindex(data.index.drop('Postcode'))
    df_all.merge(data, how='left', left_index=True, right_index=True)
    
    
    print('Processed {fname}'.format(fname=file))

In [ ]:
postcode = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/dt.csv', header = None)

In [ ]:
postcode

In [ ]:
postcode = postcode.set_index([0])

In [ ]:
data = pd.concat([labels,postcode])

In [ ]:
data

In [ ]:
data.columns = data.iloc[0]

In [ ]:
data = data.reindex(data.index.drop('Postcode'))

In [ ]:
data

In [ ]:
df_all = df_all.merge(data, how='left', left_index=True, right_index=True)

In [ ]:
df_all

In [ ]:
postcode = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/dt.csv', header = None)
postcode = postcode.set_index([0])
data = pd.concat([labels,postcode])
data.columns = data.iloc[0]
data = data.reindex(data.index.drop('Postcode'))


In [ ]:
data

In [ ]:
data.insert(0, 'date', 'NaN')

In [ ]:
data['date']= pd.to_datetime(data['date'])

In [ ]:
df_all[df_all.isnull()] = data

In [ ]:
df_all

In [ ]:
df_all.to_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/test.csv')